<a href="https://colab.research.google.com/github/joenvihe/ejemplos_machine_learning/blob/master/tesis_products_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Conexión al Google Drive**

Nos conectamos al google drive donde tenemos nuestra fuente de información

In [1]:
# Nos conectamos al google drive para obtner el dataset
from google.colab import drive
# obtener la autorizacion
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# nos ubicamos en la carpeta donde se encuentra el DATASET
%cd /content/drive/'My Drive'/UNIR/TESIS

/content/drive/My Drive/UNIR/TESIS


## **Instalación de módulos**

Instalamos todos los modulos necesarios de python para el desarrollo de la tesis

In [3]:
!pip install translate
!python -m spacy download es_core_news_sm

     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 163kB 17.9MB/s 
     |████████████████████████████████| 3.3MB 52.7MB/s 
     |████████████████████████████████| 102kB 29.3MB/s 
  Created wheel for nodeenv: filename=nodeenv-1.3.3-cp36-none-any.whl size=21349 sha256=342b3234380628b3b5d2bac9277c23be375364231cc54a7bc512e1a88e1f1b78
  Stored in directory: /root/.cache/pip/wheels/7b/6c/23/eb26369b77904c8963fae9e64338b0f0b948b4d59710760834
Successfully built nodeenv
ERROR: pytest 3.6.4 has requirement pluggy<0.8,>=0.5, but you'll have pluggy 0.12.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
     |████████████████████████████████| 11.1MB 759kB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.1.0-cp36-none-any.whl size=11111557 

In [4]:
############################################################################################
# Herramientas varias
############################################################################################
import pandas as pd
import numpy as np
import string
import re
import collections
from collections import Counter
from pprint import pprint
############################################################################################
# Herramientas de traductor
############################################################################################
from translate import Translator
############################################################################################
# Herramientas de spacy y procesaminto de lenguaje natural
############################################################################################
import spacy
import es_core_news_sm
from spacy import displacy
############################################################################################
# Herramientas de NLTK y procesaminto de lenguaje natural
############################################################################################
import nltk
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,SnowballStemmer
from nltk.corpus import wordnet as wn
############################################################################################
# Algoritmo de clustering k-mean
############################################################################################
from sklearn.cluster import KMeans, MiniBatchKMeans,AgglomerativeClustering,DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.feature_extraction.text import TfidfVectorizer
############################################################################################
# Herramientas de gensim y procesaminto de lenguaje natural
############################################################################################
from gensim import corpora
from gensim.models import LsiModel
from gensim.models import Word2Vec
from gensim.models import Phrases
from gensim.models.coherencemodel import CoherenceModel
############################################################################################
# Herramientas de graficos
############################################################################################
import matplotlib.pyplot as plt
############################################################################################
# Herramientas de scrapping
############################################################################################
import requests
from bs4 import BeautifulSoup

#descargas necesarias para NLTK
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw')
nltk.download('wordnet')

#inicio de objeto spacy
nlp = es_core_news_sm.load()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


## **Lectura de DataSet de productos de Supermercado**

Se obtiene el dataset de los productos de supermercados. 

In [0]:
# cargamos en un dataframe el dataset de los productos de supermercados remplazando los valores nulos por espacios en blanco
df = pd.read_csv("web_products.csv")
df = df.replace(np.nan, '', regex=True)

In [0]:
# se visualiza el contenido del dataframe
df.head(30)

,pid,cat3,web,brand,pname
0,14610,chocolates de leche,wong,orquidea,chocolate orquidea x 85 gr leche c/coco - cho...
1,10643,crema dental,plaza_vea,oral-b,crema dental oral-b stages princess + cepillo ...
2,40669997,limpiadores,tottus,tottus,sacagrasa aroma naranja
3,40308095,galletas dulces,tottus,costa,galletas chocodonuts crunch six pack
4,5470,bebidas rehidratantes,plaza_vea,powerade,bebida rehidratante powerade multifrutas botel...
5,9911,gomitas y masticables,vivanda,mentos,goma de mascar mentos pure sin azúcar frasco 56gr
6,6146,vino tinto,plaza_vea,trapiche,vino trapiche varietal malbec botella 750 ml
7,46094,,plaza_vea,ferymar,ají mirasol ferymar
8,3421,galletas,wong,nabisco,galletas oreo nabisco cookies and cream pack 6...
9,1321,galletas dulces,plaza_vea,nestlé,galletas morochas bañadas con pasta sabor a ch...


## **Limpieza del nombre de producto**

Se desarrolla una función donde se limpia el nombre del producto, se retira la marca, el nombre de la web, la unidad de medida, valores numéricos, etc.
Se adiciona una columna al dataframe con el nombre de clean product, donde se encuentra el nombre del producto limpio.

In [0]:
lst_medida = ["dt","gr","un","kl","kg","ml","mt","cm","oz","oi","plz","bl","unidades","litro","und","unid","talla","ts","ch","pack","lata","latas","bolsa","bolsas","botella","botellas","paquete","paquetes","caja","frasco","frascos"]

# se crea una funcion para limpiar el nombre del producto
def clean_product(web,brand,name_prod):
  #name_prod = translate(name_prod)
  # se retira el nombre de la web
  name_prod = str(name_prod).replace(str(web)," ") 
  # se retira la marca
  name_prod = str(name_prod).replace(str(brand)," ") 
  # se retira los signos de puntuacion
  for i in string.punctuation:
    name_prod = str(name_prod).replace(i," ")
  # se retira los numeros
  name_prod = re.sub('x\d', ' ', name_prod)
  name_prod = re.sub('\d', ' ', name_prod)
  # se crea una bolsa de palabras sin stopword y unidades de medidas
  bag_word = ""
  for i in name_prod.split(" "):
    if i not in stopwords.words('spanish') and i not in lst_medida and len(i)>1:
    #if i not in lst_medida and len(i)>1:
      bag_word = bag_word + " " + str(i)
  # se valida que cada palabra de la bolsa tenga al menos un caracter
  lst_word = []
  for i in bag_word.split(" "):
    if len(i)>1 and i not in lst_word:
      lst_word.append(i)
      
  return " ".join(lst_word)

In [0]:
# se genera la función para crear el campo clean_product
df['clean_product'] = df.apply(lambda x: clean_product(x['web'],x['brand'],x['pname']),axis=1)

In [0]:
df.head()

,Unnamed: 0,pid,cat3,web,brand,pname,clean_product
0,0,14610,chocolates de leche,wong,orquidea,chocolate orquidea x 85 gr leche c/coco - cho...,chocolate leche coco orqui
1,1,10643,crema dental,plaza_vea,oral-b,crema dental oral-b stages princess + cepillo ...,crema dental stages princess cepillo suave
2,2,40669997,limpiadores,tottus,tottus,sacagrasa aroma naranja,sacagrasa aroma naranja
3,3,40308095,galletas dulces,tottus,costa,galletas chocodonuts crunch six pack,galletas chocodonuts crunch six
4,4,5470,bebidas rehidratantes,plaza_vea,powerade,bebida rehidratante powerade multifrutas botel...,bebida rehidratante multifrutas


In [0]:
# se graba una nueva version del dataset con el campo clean_product
df.to_csv("web_clean_product.csv")

## **Desarrollo del Agrupamiento de Productos**

Se agruparan los productos a traves de las palabras que lo forman y por la frecuencia de las mismas en las oraciones

In [0]:
# cargamos en un dataframe
df = pd.read_csv("web_clean_product.csv")
df = df.replace(np.nan, '', regex=True)
df_doc = df[["pid","cat3","web","brand","pname","clean_product"]]

#df = pd.read_csv("web_clean_product_batch_cluster_450_v1.csv")
#df = df.replace(np.nan, '', regex=True)
#df_doc = df[["pid","cat3","web","brand","pname","clean_product","cluster"]]
#df_doc = df_doc.loc[df["cluster"]==19]
#df_doc = df_doc[["pid","cat3","web","brand","pname","clean_product"]]


df_doc.head()

,pid,cat3,web,brand,pname,clean_product
0,14610,chocolates de leche,wong,orquidea,chocolate orquidea x 85 gr leche c/coco - cho...,chocolate leche coco orqui
1,10643,crema dental,plaza_vea,oral-b,crema dental oral-b stages princess + cepillo ...,crema dental stages princess cepillo suave
2,40669997,limpiadores,tottus,tottus,sacagrasa aroma naranja,sacagrasa aroma naranja
3,40308095,galletas dulces,tottus,costa,galletas chocodonuts crunch six pack,galletas chocodonuts crunch six
4,5470,bebidas rehidratantes,plaza_vea,powerade,bebida rehidratante powerade multifrutas botel...,bebida rehidratante multifrutas


In [0]:
def word_tokenizer(text):
    #tokenizes and stems the text
    #doc = nlp(u""+text)
    tokens = word_tokenize(text)  
    #tokens = [token.lemma_ for token in doc]
    tokens = [t for t in tokens if t not in stopwords.words('spanish') and len(t.strip())>0]
    #stemmer = PorterStemmer()
    stemmer = SnowballStemmer('spanish')
    tokens = [stemmer.stem(t) for t in tokens]
    return tokens
    #return lst_token 
    
    
def cluster_sentences(sentences, nb_of_clusters=5):
  tfidf_vectorizer = TfidfVectorizer(tokenizer = word_tokenizer,
                                     #stop_words = stopwords.words('spanish'),
                                     encoding = u'utf-8',
                                     ngram_range = (1, 3),
                                     #ngram_range = (1, 4),
                                     #strip_accents = 'unicode', 
                                     lowercase = True)
  #builds a tf-idf matrix for the sentences
  tfidf_matrix = tfidf_vectorizer.fit_transform(sentences)
  
   
  
  #kmeans = KMeans(n_clusters=nb_of_clusters, init="k-means++" , n_init = 10, max_iter = 100, random_state=550,n_jobs =30,algorithm = "full") #700 ,rs=550, 2444
    
  kmeans = KMeans(n_clusters=nb_of_clusters, init="k-means++" , n_init = 10, max_iter = 100, random_state=550,n_jobs =30,algorithm = "full") #550 600 #2552 
  
  #kmeans = KMeans(n_clusters=nb_of_clusters, init="k-means++" , n_init = 10, max_iter = 100, random_state=400,n_jobs =30,algorithm = "full") #450 #3652
  
  
  
  kmeans.fit(tfidf_matrix)
  clusters = collections.defaultdict(list)
  for i, label in enumerate(kmeans.labels_):
    clusters[label].append(i)
  return dict(clusters)

In [0]:
#df_doc.columns
s_pid = df_doc["pid"].values
s_cat3 = df_doc["cat3"].values
s_web = df_doc["web"].values
s_brand = df_doc["brand"].values  
s_product = df_doc["pname"].values  
s_clean_product = df_doc["clean_product"].values # sin marca y sin web 

In [0]:
lst_product_cluster = [] 
nclusters= 650
clusters = cluster_sentences(s_clean_product, nclusters)

In [0]:
for cluster in range(nclusters):
    #print("cluster ",cluster,":")
    for i,sentence in enumerate(clusters[cluster]):
        #print("    ",s_clean_product[sentence]," - ",s_web[sentence])
        dict_cluster = {}
        dict_cluster["pid"] = s_pid[sentence]
        dict_cluster["cat3"] = s_cat3[sentence]
        dict_cluster["web"] = s_web[sentence]
        dict_cluster["brand"] = s_brand[sentence] 
        dict_cluster["pname"] = s_product[sentence]
        dict_cluster["clean_product"] = s_clean_product[sentence] 
        dict_cluster["cluster"] = cluster
        lst_product_cluster.append(dict_cluster)
        
pd_cluster = pd.DataFrame(lst_product_cluster) 
pd_cluster.to_csv("web_clean_product_batch_cluster_650_v1.csv") # modificar el ngrama range

## **Calculo de palabras en comun**

Se desarrolla una función para calcular el porcentaje de similaridad entre palabras de una oración

In [0]:
def porcentaje_palabras_comun(pal1,pal2): 
  # initialize regex tokenizer
  en_stop = set(stopwords.words('spanish'))
  tokenizer = RegexpTokenizer(r'\w+')
  
  tokensA = tokenizer.tokenize(pal1)
  tokensA = [i for i in tokensA if not i in en_stop]
  
  tokensB = tokenizer.tokenize(pal2)
  tokensB = [i for i in tokensB if not i in en_stop]
  
  
  tokens = list(set(tokensA) & set(tokensB))
  
  palA = ""
  for i in tokensA:
    if i not in tokens:
      palA += " " + i
      
  palB = ""
  for i in tokensB:
    if i not in tokens:
      palB += " " + i
  
  return (len(tokens)/len(tokensA))*100, palA, palB

In [0]:
# cargamos en un dataframe
df = pd.read_csv("web_clean_product_batch_cluster_650_v1.csv")
df = df[["brand","cat3","clean_product","cluster","pid","pname","web"]] 

Ejemplo colocando el numero de CLUSTER

In [24]:
df_producto = df.loc[df["cluster"]==14]
df_producto.head(50)

,brand,cat3,clean_product,cluster,pid,pname,web
277,el olivar,aliños y vinagretas,vinagreta light mostaza,14,3981,vinagreta el olivar light a la mostaza botella...,plaza_vea
278,biosal,"sal, sal parrillera y sal especial",sal light,14,7660,sal light biosal frasco 125 g,wong
279,gloria,jugos,jugo durazno light,14,229,jugo de durazno gloria light caja 1 l,metro
280,bonle,queso fresco,queso fresco light,14,10170316,queso fresco light,tottus
281,blue bonnet,margarina,margarina light pote,14,4783,margarina blue bonnet light pote 425g,plaza_vea
282,vivanda,carne molida de res,vacuno carne molida light bandeja aprox,14,8951,vacuno carne molida light (bandeja = 400gr aprox),vivanda
283,maybelline,rostro,corrector mny better sk concealer light cor,14,14347,corrector mny better sk concealer light - cor...,wong
284,dorina,margarina en pote,margarina light,14,20068726,margarina light,tottus
285,dorina,margarina,margarina light pote,14,4299,margarina dorina light pote 900g,plaza_vea
286,ocean spray,jugos líquidos,jugo sabor cranberry light,14,41011028,jugo con sabor a cranberry light,tottus


In [0]:
lst_prod =[]
for i,row1 in df_producto.iterrows():
  for j,row2 in df_producto.iterrows():
    if row1["clean_product"] != row2["clean_product"]:
      
      porc, palA, palB = porcentaje_palabras_comun(row1["clean_product"],row2["clean_product"])
      porc2, palA2, palB2 = porcentaje_palabras_comun(row1["pname"],row2["pname"])
      
      porc3, palB3, palA3 = porcentaje_palabras_comun(row2["clean_product"],row1["clean_product"])
      porc4, palB4, palA4 = porcentaje_palabras_comun(row2["pname"],row1["pname"])
      
      porc_med = ((porc+porc2+porc3+porc4)/4)
      
      if porc_med > 60 :
        dict_prod = {}
        dict_prod["pp_brand"] = row1["brand"]
        dict_prod["pp_cat3"] = row1["cat3"]
        dict_prod["pp_clean_product"] = row1["clean_product"]
        dict_prod["pp_cluster"] = row1["cluster"]
        dict_prod["pp_pid"] = row1["pid"]
        dict_prod["pp_pname"] = row1["pname"]
        dict_prod["pp_web"] = row1["web"] 
        dict_prod["porc_sim_palabra"] = porc
        dict_prod["porc_sim_palabra2"] = porc2
        dict_prod["porc_sim_palabra3"] = porc3
        dict_prod["porc_sim_palabra4"] = porc4
        dict_prod["porc_sim_palabra_prom"] = porc_med
        dict_prod["ps_brand"] = row2["brand"]
        dict_prod["ps_cat3"] = row2["cat3"]
        dict_prod["ps_clean_product"] = row2["clean_product"]
        dict_prod["ps_cluster"] = row2["cluster"]
        dict_prod["ps_pid"] = row2["pid"]
        dict_prod["ps_pname"] = row2["pname"]
        dict_prod["ps_web"] = row2["web"] 
        dict_prod["pp_dif_pal"] = palA
        dict_prod["ps_dif_pal"] = palB
        dict_prod["pp_dif_pal2"] = palA2
        dict_prod["ps_dif_pal2"] = palB2
        dict_prod["pp_dif_pal3"] = palA3
        dict_prod["ps_dif_pal3"] = palB3
        dict_prod["pp_dif_pal4"] = palA4
        dict_prod["ps_dif_pal4"] = palB4
        lst_prod.append(dict_prod)
  #break
  
df_sp = pd.DataFrame(lst_prod)

df_sp.head()
df_sp.to_csv("web_producto_clean_cluster_porc_sim_v11.csv")


In [27]:
df_sp[["porc_sim_palabra_prom","pp_brand","pp_pid","pp_pname","pp_web","ps_brand","ps_pid","ps_pname","ps_web"]]

,porc_sim_palabra_prom,pp_brand,pp_pid,pp_pname,pp_web,ps_brand,ps_pid,ps_pname,ps_web
0,80.000000,bonle,10170316,queso fresco light,tottus,gloria,41509191,queso fresco light molde pequeño,tottus
1,80.000000,bonle,10170316,queso fresco light,tottus,gloria,41014129,queso fresco light - molde grande,tottus
2,66.666667,bonle,10170316,queso fresco light,tottus,president,41255557,slice queso light,tottus
3,75.000000,blue bonnet,4783,margarina blue bonnet light pote 425g,plaza_vea,dorina,20068726,margarina light,tottus
4,75.000000,blue bonnet,4783,margarina blue bonnet light pote 425g,plaza_vea,dorina,20068723,margarina light,tottus
5,73.214286,vivanda,8951,vacuno carne molida light (bandeja = 400gr aprox),vivanda,vivanda,32168,carne molida light,vivanda
6,60.714286,vivanda,8951,vacuno carne molida light (bandeja = 400gr aprox),vivanda,plaza vea,9343,carne molida light de res,plaza_vea
7,75.000000,dorina,20068726,margarina light,tottus,blue bonnet,4783,margarina blue bonnet light pote 425g,plaza_vea
8,76.666667,dorina,20068726,margarina light,tottus,dorina,4299,margarina dorina light pote 900g,plaza_vea
9,76.666667,dorina,20068726,margarina light,tottus,dorina,4736,margarina dorina light pote 450g,plaza_vea
